This notebook will use the tools we have created to see what the optimal counter-army is to the Terran Bio build, which revolves around biological units supported by Medivacs. The biological Terran units include the Marine, Marauder, Reaper, and Ghost. Reapers and Ghosts are mainly used for harassment and not large scale battles. A typical Terran Bio build has Marines and Marauders with a ratio around 3:1, and Medivacs with a ratio around 1 Medivac for every 10 or so infantry units. Two common ratios of Marine:Marauder:Medivac are 6:2:1 and 15:5:2. For our purposes, we will be testing the 15:5:2 ratio and optimizing against 45 Marines, 15 Marauders, and 6 Medivacs. This army has a total supply of 87, simulating an early- to mid-game composition. By default, our random army generator creates armies with total supply ranging anywhere between 0 and 200. To reduce the amount of iterations needed to hone in on the optimal solution, we will be limiting our random armies to a supply cap of 120, since we know for certain that pretty much any army with a supply greater than that will be suboptimal (such armies will likely be overkill, or contain too many support units that do not add enough value to the overall army composition). It is likewise reasonable to raise the floor of our random supply cap range from 0 to 20 since we know for a fact that no army with less than 20 supply can defeat our Terran Bio army in a reasonable amount of time.

The Terran Bio build is standard because it revolves around Marines which are cheap, one of the first units Terran players can produce, and are extremely versatile as they are fast, ranged, and can attack both ground and air units. Marauders act as meat-shields in addition to extra damage against armored units, and Medivacs provide constant healing to both Marines and Marauders. While the functionality is not included in our combat simulator, Medivacs can also carry a small group of Marines and quickly fly them around the map, making this build useful for harassment in addition to all-out battle.

In [ ]:
from sim_units import get_Units, get_Terran, get_Protoss, get_Zerg
from generate_comps import get_army_supply, init_army_comps
from linear_program import find_optimal_army
import json
from joblib import Parallel, delayed
import time

In [ ]:
def find_counters_parallel(army_comp, file_name, race, supply_cap=200, min_supply=0, num_tests=10):
    """
    army_comp is the enemy army composition we are optimizing against
    file_name is a string, name of txt file we will save our results to
    race:{'Terran', 'Protoss', 'Zerg'} the race we are optimizing for
    supply_cap is an integer of the largest army we wish to test
    num_tests is an integer, the number of times we wish to run our test
    Creates a file with the army counters
    """
    army_list = []
    for i in range(num_tests):
        army_list.append(army_comp)
    start_time = time.time()
    terran_counters = {}
    results = Parallel(n_jobs=3)(delayed(find_optimal_army)(enemy, race, supply_cap=supply_cap, min_supply=min_supply)
                             for enemy in army_list)
    run_time = time.time() - start_time
    print(run_time)
    with open(file_name, 'a') as file:
        file.write("Enemy army:")
        file.write('\n')
        file.write(str(army_comp))
        file.write('\n')
        file.write('\n')
        file.write(str(run_time))
        file.write('\n')
        file.write('\n')
        file.write("Counters:")
        for line in results:
            file.write(line)
            file.write('\n')
    return results

In [ ]:
terran_bio = {}
terran_bio['Marine'] = 45
terran_bio['Marauder'] = 15
terran_bio['Medivac'] = 6

In [ ]:
terran_bio_counters = find_counters_parallel(terran_bio, 'terran_bio.txt', 'Terran', supply_cap=125, min_supply=20)